In [1]:
date = 20220930

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

HOME_NA = pd.read_excel(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\05_Home_Financing\03_Newly_Approved\\'+str(date)+'\\NA_'+str(date)+'.xlsx')


HOME_NA['Product_Code'] = HOME_NA['Product_Code'].str.strip()
HOME_NA['Product_Code'] = HOME_NA['Product_Code'].str.upper()

HOME_NA.Approved_Amt.fillna(-9999,inplace=True)

HOME_MIB_NA = HOME_NA.iloc[np.where(HOME_NA.Product_Code.isin((['H2','H13','H19','H22','H93','H94','L1','L15','L16','L17','L18','L20','L25','L26','W2','W9','W13','W14','W56','W65','T43'])) & (HOME_NA.Approved_Amt != -9999))]

HOME_MIB_NA['NOA'] = 1

HOME_MIB_NA['OSM'] = HOME_MIB_NA.Approved_Amt/1000000

def RISK_RATING(Rating):
    if Rating==1:
        return '01. R1'
    elif Rating==2: 
        return '02. R2'
    elif Rating==3: 
        return '03. R3'
    elif Rating==4: 
        return '04. R4'
    elif Rating==5: 
        return '05. R5'
    elif Rating==6:
        return '06. R6'
    elif Rating==7: 
        return '07. R7'
    elif Rating==8: 
        return '08. R8'
    elif Rating==9: 
        return '09. R9'
    elif Rating==10: 
        return '10. R10'
    elif Rating==11: 
        return '11. R11'
    else:
        return '10. Not Populated'
HOME_MIB_NA['RISK_RATING']= HOME_MIB_NA.Rating.apply(RISK_RATING)

def INCOME_BRACKET(Gross_Income):
    if Gross_Income <= 3000:
        return '01. Up to RM3,000'
    elif Gross_Income > 3000 and Gross_Income <= 5000:
        return '02. Above RM3,000 to RM5,000'
    elif Gross_Income > 5000 and Gross_Income <= 8000:
        return '03. Above RM5,000 to RM8,000'
    elif Gross_Income > 8000 and Gross_Income <= 15000:
        return '04. Above RM8,000 to RM15,000'
    elif Gross_Income > 15000 and Gross_Income <= 20000:
        return '05. Above RM15,000 to RM20,000'
    elif Gross_Income > 20000:
        return '06. Above RM20,000'
    else:
        return '07. Not Populated'
HOME_MIB_NA['INCOME_BRACKET'] = HOME_MIB_NA.Gross_Income.apply(INCOME_BRACKET)

def PROPERTY_VALUE(Property_Purchase_Price):
    if Property_Purchase_Price > 0 and Property_Purchase_Price <= 100000:
        return '01. Up to RM100K'
    elif Property_Purchase_Price > 100000 and Property_Purchase_Price <= 150000:
        return '02. Above RM100K to RM150K'
    elif Property_Purchase_Price > 150000 and Property_Purchase_Price <= 250000: 
        return '03. Above RM150K to RM250K'
    elif Property_Purchase_Price > 250000 and Property_Purchase_Price <= 500000:
        return '04. Above RM250K to RM500K'
    elif Property_Purchase_Price > 500000:
        return '05. Above RM500K';
    else:
        return '06. Not Populated';
HOME_MIB_NA['PROPERTY_VALUE'] = HOME_MIB_NA.Property_Purchase_Price.apply(PROPERTY_VALUE)

HOME_MIB_NA.State_Name = HOME_MIB_NA.State_Name.str.strip()

HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Kuala Lumpur FT','Labuan FT']),'STATE']='01. Federal Territory'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Johor']),'STATE']='02. Johor'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Kedah']),'STATE']='03. Kedah'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Kelantan']),'STATE']='04. Kelantan'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Melaka']),'STATE']='05. Melaka'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Negeri Sembilan']),'STATE']='06. N.Sembilan'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Pahang']),'STATE']='07. Pahang'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Penang']),'STATE']='08. Penang'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Perak']),'STATE']='09. Perak'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Perlis']),'STATE']='10. Perlis'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Sabah']),'STATE']='11. Sabah'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Sarawak']),'STATE']='12. Sarawak'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Selangor']),'STATE']='13. Selangor'
HOME_MIB_NA.loc[HOME_MIB_NA.State_Name.isin(['Terengganu']),'STATE']='14. Terengganu'
HOME_MIB_NA.STATE.fillna(-9999,inplace=True)
HOME_MIB_NA.loc[HOME_MIB_NA.STATE.isin([-9999]),'STATE']='15. Not Populated'

def MOF(Property_Purchase_Price,Approved_Amt,Property_OMV):
    if Property_Purchase_Price > 0 :
        return Approved_Amt/Property_Purchase_Price
    else:
        return Approved_Amt/Property_OMV
HOME_MIB_NA['MOF'] = HOME_MIB_NA.apply(lambda x: MOF(x['Property_Purchase_Price'], x['Approved_Amt'], x['Property_OMV']), axis=1)

def MOF_Band(MOF):
    if MOF < 0.70:
        return "01.<70"
    elif MOF < 0.80:
        return "02.70 TO <80"
    elif MOF < 0.85:
        return "03.80 TO <85"
    elif MOF < 0.90:
        return "04.85 TO <90"
    elif MOF > 0.90:
        return "05.>=90"
    else:
        return "06. Not Populated"
HOME_MIB_NA['MOF_Band'] = HOME_MIB_NA.MOF.apply(MOF_Band)

def DSR_Band(DSR):
    if DSR > 0  and DSR < 50:
        return "01.<50"
    elif DSR >= 50 and DSR < 60: 
        return "02.50-<60"
    elif DSR >= 60 and DSR < 70:
        return "03.60-<70"
    elif DSR >= 70 and DSR < 80:
        return "04.70-<80"
    elif DSR >= 80 and DSR < 90:
        return "05.80-<90"
    elif DSR >= 90 and DSR < 100:
        return "06.90-<100"
    elif DSR >= 100:
        return"07.>=100"
    else:
        "08.NA"
HOME_MIB_NA['DSR_Band']=HOME_MIB_NA.DSR.apply(DSR_Band)

row = ['OSM','NOA']


RISK_RATING1 = HOME_MIB_NA.groupby(['RISK_RATING'])[row].sum().reset_index()
RISK_RATING2 = HOME_MIB_NA.groupby(['INCOME_BRACKET'])[row].sum().reset_index()
RISK_RATING3 = HOME_MIB_NA.groupby(['PROPERTY_VALUE'])[row].sum().reset_index()
RISK_RATING4 = HOME_MIB_NA.groupby(['STATE'])[row].sum().reset_index()
RISK_RATING5 = HOME_MIB_NA.groupby(['MOF_Band'])[row].sum().reset_index()
RISK_RATING6 = HOME_MIB_NA.groupby(['DSR_Band'])[row].sum().reset_index()

writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\05_Home_Financing\04_Dataset\\'+str(date)[:6]+'\HF_NA'+str(date)[:6]+'(python).xlsx',engine='xlsxwriter')

RISK_RATING1.to_excel(writer2, sheet_name='NA_RISK_RATING', index = False)
RISK_RATING2.to_excel(writer2, sheet_name='NA_INCOME_BRACKET', index = False)
RISK_RATING3.to_excel(writer2, sheet_name='NA_PROPERTY_VALUE', index = False)
RISK_RATING4.to_excel(writer2, sheet_name='NA_STATE', index = False)
RISK_RATING5.to_excel(writer2, sheet_name='NA_MOF_Band', index = False)
RISK_RATING6.to_excel(writer2, sheet_name='NA_DSR_Band', index = False)

writer2.save()

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h